# Reinforcement Learning для игры крестиков ноликов

Попробуем применить Reinforcement learnign на простой игре. Наприме: крестики-нолики.

In [1]:
import numpy as np
import pandas as pd
from keras.layers import Dense, Conv2D, Input, Flatten, Dropout, Activation
from keras.models import Sequential
from keras.models import Model
from random import random
import uuid
import os
import copy
from sklearn.model_selection import train_test_split

D:\ProgrammFiles\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [13]:

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# Общее описание алгоритма

Пусть у нас есть модель, которая играет случайным образом. Мы проводим N партий между случайными моделями. Тем самым мы генерируем набор данных. Пусть мы находимся в состоянии s, и делаем ход, который переводит нас в состояние s_1. Состояние s_1 мы запоминаем до окончании партии. Пусть в конце партии у игрока 1 есть множество состояний {s_i_1}, а у игрока 2 множество {s_j_2} и предположим, что игрок 1 победил. Тогда будет сформирован набор данных, в которых всем целевым переменным состояния из множества {s_i_1} присвоенно 1, а всем целевым переменным из множества {s_i_2} 0. При ничейном результате можно писать во все состояния 0,5. Затем обучаем модель на этих данных. Теперь проводим партии между этой моделью и предыдущей, тем самым формируя новый набор данных. И та модель которая окажется сильнее переходит в следующий раунд.  
Обобщая, в каждый момент времени у нас есть лучшая модель. Мы обучаем новую модель на данных и проводим партии между ней и лучшей моделью, формируя новые данные и отбирая лучшую модель

In [14]:
file_path = './tic_toc_dataset/'

In [15]:
f = open(file_path + "dataset.csv", "w")

In [16]:
f.close()

# Механика игры

Для начала необходимо реализовать саму механику игры крестики-нолики.  
Класс Tic управляет всей игрой.  
Конструктор данного класса пинимает два параметра типа Player.  
Функция start() инициализирует начало игры (поле 3 на 3 заполненое нулями, кто первый ходит и запускает цикл игры)  
Функция move() запрашивает ход у player, проверяет на окончание игры. В случае окончания игры, вызывается функция окончания у player.

In [17]:
class Tic:    
    def __init__(self, player1, players2):
        self.players = [player1, player2]
        
    def start(self):        
        self.feed = np.zeros([3, 3], dtype=int)      
        self.step_player = 0
        self.end = False
        self.result = ''
        while self.end == False:
            self.move()      
        return self.result
        
    def check_win(self, x, y):
        if self.feed[0][y] == self.feed[1][y] == self.feed[2][y]:
            return True
        if self.feed[x][0] == self.feed[x][1] == self.feed[x][2]:
            return True        
        if x == y and self.feed[0][0] == self.feed[1][1] == self.feed[2][2]:
            return True
        if x + y == 2 and self.feed[0][2] == self.feed[1][1] == self.feed[2][0]:
            return True
        return False   
    
    def check_end(self):
        for i in self.feed:
            for cell in i:
                if cell == 0:
                    return False
        return True
    
    def move(self):        
        i, j = self.players[self.step_player].do_step(self.feed, self.step_player + 1)        
        if self.feed[i][j] == 0:            
            self.feed[i][j] = self.step_player + 1                    
            if self.check_win(i, j):
                self.players[self.step_player].end("win")
                self.result = str(self.step_player)
                self.players[0 if self.step_player == 1 else 1].end("lose")
                self.end = True
                return            
            if self.check_end():
                self.players[self.step_player].end("draw")
                self.result = 'draw'
                self.players[0 if self.step_player == 1 else 1].end("draw")
                self.end = True
                return
            self.step_player = 0 if self.step_player == 1 else 1
            

# Архитектура

Возможная архитектура нейронной сети. Данный код пока что нигде не использутеся.

In [18]:
inp = Input(shape=(3, 3, 3))

conv = Conv2D(9, (3, 3))(inp)

flat = Flatten()(conv)
dense = Dense(18, activation='relu')(flat)
drop = Dropout(0.1)(dense)
out = Dense(1, activation='softmax')(drop)

model = Model(inputs = [inp], outputs = [out])
model.compile(loss = 'mean_squared_error', optimizer='adam', metrics=['mae', 'acc'])

In [19]:
model = Sequential()
model.add(Conv2D(9, (3, 3), padding='same',
                 input_shape=(3, 3, 3)))
model.add(Activation('relu'))
#model.add(Dropout(0.1))
model.add(Flatten())
model.add(Dense(18))
model.add(Activation('relu'))
#model.add(Dropout(0.1))
model.add(Dense(1))
#model.add(Activation('softmax'))
model.compile(loss = 'mean_squared_error', optimizer='adam')

Опишем класс Player.  
Конструктор данного класса принимает модель. Которая умеет по состояния игры отвечать, на сколько данная ситуация хорошая.  
В классе Player содержится метод do_step(), который вызывается из класса Tic (см. выше)  
В этом методе происходит следующее:  
1. мы смотрим все возможные ходы.
2. состояния, которые получаться после каждого возможного хода мы подаем в модель, которая говорит насколько этот ходы хороший.
3. выбираем и храним лучший ход.
4. все сделанные ходы в данной партии мы храним в self.steps (когда мы делаем ход мы не знаем к чему он привидет)  
Метод end() вызывется, когда закончится партия. В этом методе необходимо будет записать все ходы в некое хранилище с резуьтатом данной партии. 

In [20]:
class Player:
    def __init__(self, model):        
        self.model = model
        self.steps = []
    def do_step(self, feed, number):        
        #top_x = -1
        #top_y = -1
        #top_value = 0
        #top_step = np.zeros((3, 3, 3), dtype=int)        
        values = []
        xs = []
        ys = []
        steps_poss = []       
        for x in range(3):
            for y in range(3):                
                if feed[x][y] != 0:
                    continue
                step = np.zeros((3, 3, 3), dtype=int)
                for i in range(3):
                    for j in range(3):
                        step[0][i][j] = 1 if feed[i][j] == 1 else 0
                        step[1][i][j] = 1 if feed[i][j] == 2 else 0
                        step[2][i][j] = number - 1
                step[0 if number == 1 else 1][x][y] = 1                
                value = self.model.predict(step.reshape(1, 3, 3, 3))[0]                          
                if type(value) != float:                    
                    value = value[0]                
                if value < 0:
                    value = 0                            
                values.append(value)
                xs.append(x)
                ys.append(y)
                steps_poss.append(step)
                #if value > top_value:                    
                    #top_value = value
                    #top_x = x
                    #top_y = y
                    #top_step = np.copy(step)                        
        #self.steps.append(top_step)        
        values = np.array(values)
        s = values.sum()
        values = values / s        
        index = np.random.choice(len(values), p = values)
        self.steps.append(np.copy(steps_poss[index]))        
        return xs[index], ys[index]
        #return top_x, top_y
    def end(self, status):
        np_steps = np.reshape(self.steps, (len(self.steps), 27))
        df = pd.DataFrame(np_steps)
        answer = 0
        if status == 'win':
            answer = 1
        if status == 'draw':
            answer = 0.5
        df.insert(27, '27', answer)        
        with open(file_path+'dataset.csv', 'a') as f:
            df.to_csv(f, header=False)        
        f.close()
        self.steps = []

Первая модель самая примитивня: возвращает случайное число.

In [21]:
## model for random
class Model_random:
    def __init__(self):
        pass
    def predict(self, a):
        return [random()]

# Запуск

Пробуем запустить 2-ух игроков со случайными моделями

In [22]:
f = open(file_path + "dataset.csv", "w")
f.close()

In [23]:
player_top = Player(Model_random())
player2 = Player(Model_random())
tic = Tic(player_top, player2)
for x in range(100):
    tic.start() 

In [24]:
for x in range(10):    
    df = pd.read_csv(file_path + 'dataset.csv', header=None, usecols=[x for x in range(1, 29)])
    if (len(df) > 1000):
        df = df[len(df)-1000: len(df)]
        f = open(file_path + "dataset.csv", "w")
        df.to_csv(f, header=False) 
        f.close()       
    gr = df.groupby([x for x in range(1, 28)], as_index=False)
    dataset = gr.agg({28: np.mean})
    y_train_test = dataset.iloc[::, -1::]
    x_train_test = dataset.iloc[::,:-1:]
    x_train_test = x_train_test.values.reshape((len(x_train_test), 3, 3, 3))
    y_train_test = y_train_test.values
    X_train, X_test, y_train, y_test = train_test_split(x_train_test, y_train_test, test_size = 0.2)
    model.compile(loss = 'mean_squared_error', optimizer='adam')
    model.fit(X_train, y_train, batch_size=32, epochs=30, validation_data=(X_test, y_test), shuffle=True)
    player_new = Player(model)
    score_top = 0.0
    score_new = 0.0
    tic = Tic(player_top, player_new)    
    for i in range(50):        
        res = tic.start()        
        if res == '0':
            score_top += 1
        elif res == '1':
            score_new += 1
        elif res == 'draw':
            score_top += 0.5
            score_new += 0.5
        else:
            assert()     
    player_new = Player(model)
    tic = Tic(player_new, player_top)
    for i in range(50):        
        res = tic.start()
        if res == '1':
            score_top += 1
        elif res == '0':
            score_new += 1
        elif res == 'draw':
            score_top += 0.5
            score_new += 0.5
        else:
            assert()
    if score_new > score_top:        
        player_top = Player(model)

Train on 468 samples, validate on 118 samples
Epoch 1/30
468/468 [==============================] - 1s 2ms/step - loss: 0.3246 - val_loss: 0.2589
Epoch 2/30
468/468 [==============================] - 0s 128us/step - loss: 0.2356 - val_loss: 0.2165
Epoch 3/30
468/468 [==============================] - 0s 140us/step - loss: 0.1974 - val_loss: 0.1944
Epoch 4/30
468/468 [==============================] - 0s 137us/step - loss: 0.1793 - val_loss: 0.1850
Epoch 5/30
468/468 [==============================] - 0s 157us/step - loss: 0.1694 - val_loss: 0.1793
Epoch 6/30
468/468 [==============================] - 0s 132us/step - loss: 0.1619 - val_loss: 0.1733
Epoch 7/30
468/468 [==============================] - 0s 101us/step - loss: 0.1562 - val_loss: 0.1690
Epoch 8/30
468/468 [==============================] - 0s 100us/step - loss: 0.1500 - val_loss: 0.1666
Epoch 9/30
468/468 [==============================] - 0s 96us/step - loss: 0.1458 - val_loss: 0.1655
Epoch 10/30
468/468 [==================

Epoch 20/30
595/595 [==============================] - 0s 93us/step - loss: 0.1220 - val_loss: 0.1491
Epoch 21/30
595/595 [==============================] - 0s 91us/step - loss: 0.1222 - val_loss: 0.1509
Epoch 22/30
595/595 [==============================] - 0s 93us/step - loss: 0.1201 - val_loss: 0.1499
Epoch 23/30
595/595 [==============================] - 0s 85us/step - loss: 0.1194 - val_loss: 0.1494
Epoch 24/30
595/595 [==============================] - 0s 101us/step - loss: 0.1178 - val_loss: 0.1492
Epoch 25/30
595/595 [==============================] - 0s 94us/step - loss: 0.1154 - val_loss: 0.1521
Epoch 26/30
595/595 [==============================] - 0s 95us/step - loss: 0.1136 - val_loss: 0.1481
Epoch 27/30
595/595 [==============================] - 0s 86us/step - loss: 0.1135 - val_loss: 0.1498
Epoch 28/30
595/595 [==============================] - 0s 100us/step - loss: 0.1116 - val_loss: 0.1487
Epoch 29/30
595/595 [==============================] - 0s 67us/step - loss: 0.11

576/576 [==============================] - 0s 93us/step - loss: 0.1317 - val_loss: 0.1452
Epoch 10/30
576/576 [==============================] - 0s 89us/step - loss: 0.1324 - val_loss: 0.1372
Epoch 11/30
576/576 [==============================] - 0s 87us/step - loss: 0.1254 - val_loss: 0.1414
Epoch 12/30
576/576 [==============================] - 0s 104us/step - loss: 0.1220 - val_loss: 0.1400
Epoch 13/30
576/576 [==============================] - 0s 86us/step - loss: 0.1207 - val_loss: 0.1376
Epoch 14/30
576/576 [==============================] - 0s 87us/step - loss: 0.1166 - val_loss: 0.1424
Epoch 15/30
576/576 [==============================] - 0s 87us/step - loss: 0.1182 - val_loss: 0.1385
Epoch 16/30
576/576 [==============================] - 0s 75us/step - loss: 0.1152 - val_loss: 0.1417
Epoch 17/30
576/576 [==============================] - 0s 94us/step - loss: 0.1112 - val_loss: 0.1417
Epoch 18/30
576/576 [==============================] - 0s 91us/step - loss: 0.1106 - val_loss

582/582 [==============================] - 0s 114us/step - loss: 0.0991 - val_loss: 0.1436
Epoch 29/30
582/582 [==============================] - 0s 105us/step - loss: 0.0970 - val_loss: 0.1382
Epoch 30/30
582/582 [==============================] - 0s 109us/step - loss: 0.0970 - val_loss: 0.1378
Train on 573 samples, validate on 144 samples
Epoch 1/30
573/573 [==============================] - 1s 1ms/step - loss: 0.1449 - val_loss: 0.1553
Epoch 2/30
573/573 [==============================] - 0s 106us/step - loss: 0.1329 - val_loss: 0.1587
Epoch 3/30
573/573 [==============================] - 0s 89us/step - loss: 0.1251 - val_loss: 0.1534
Epoch 4/30
573/573 [==============================] - 0s 99us/step - loss: 0.1185 - val_loss: 0.1543
Epoch 5/30
573/573 [==============================] - 0s 104us/step - loss: 0.1145 - val_loss: 0.1545
Epoch 6/30
573/573 [==============================] - 0s 110us/step - loss: 0.1104 - val_loss: 0.1564
Epoch 7/30
573/573 [=============================

D:\ProgrammFiles\anaconda3\lib\site-packages\ipykernel_launcher.py:42: RuntimeWarning: invalid value encountered in true_divide
D:\ProgrammFiles\anaconda3\lib\site-packages\ipykernel_launcher.py:43: RuntimeWarning: invalid value encountered in less


Train on 575 samples, validate on 144 samples
Epoch 1/30
575/575 [==============================] - 1s 1ms/step - loss: 0.1405 - val_loss: 0.1570
Epoch 2/30
575/575 [==============================] - 0s 102us/step - loss: 0.1255 - val_loss: 0.1488
Epoch 3/30
575/575 [==============================] - 0s 116us/step - loss: 0.1150 - val_loss: 0.1570
Epoch 4/30
575/575 [==============================] - 0s 112us/step - loss: 0.1091 - val_loss: 0.1512
Epoch 5/30
575/575 [==============================] - 0s 83us/step - loss: 0.1041 - val_loss: 0.1499
Epoch 6/30
575/575 [==============================] - 0s 100us/step - loss: 0.1016 - val_loss: 0.1496
Epoch 7/30
575/575 [==============================] - 0s 90us/step - loss: 0.0979 - val_loss: 0.1478
Epoch 8/30
575/575 [==============================] - 0s 82us/step - loss: 0.0952 - val_loss: 0.1494
Epoch 9/30
575/575 [==============================] - 0s 118us/step - loss: 0.0927 - val_loss: 0.1541
Epoch 10/30
575/575 [====================